In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeat
import metpy.calc as mpcalc
from metpy.units import units
import metpy.constants as mpconst
import sys
import xarray as xr
import standard_atmosphere as sa

from wrf import (getvar, interplevel, to_np, latlon_coords, get_cartopy,
                 cartopy_xlim, cartopy_ylim, extract_times, ALL_TIMES, interpz3d, vinterp)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mtick
%matplotlib inline

In [ ]:
#CHANGE THESE BASED ON DESIRED FILE
filetype = "fullphys" #"fullphys_expdom"

# Open the NetCDF file
path = "/home1/class/fall19/pbeatty/Documents/Nov2019Storm/wrfout_"+filetype+".nc"
ncfile = Dataset(path, "r")
timeidx = 14
# Extract the pressure, geopotential height, and wind variables
pressure = getvar(ncfile, "pressure",timeidx=timeidx)
z = getvar(ncfile, "z", timeidx=timeidx, units="m")
#hght = getvar(ncfile, "z", timeidx=timeidx)
thta = getvar(ncfile, "th",timeidx=timeidx)
tmpk = getvar(ncfile, "tk",timeidx=timeidx)
ua = getvar(ncfile, "ua", timeidx=timeidx)
va = getvar(ncfile, "va", timeidx=timeidx)

In [ ]:
levels = [1000,950,900,850,800,750,700,650,600,550,500,450,400,350,300,250,200,150,100]
levels = np.array(levels)
hght = vinterp(ncfile,z,'p',levels,extrapolate=True)
tmpk = vinterp(ncfile,tmpk,'p',levels,extrapolate=True)
thta = vinterp(ncfile,thta,'p',levels,extrapolate=True)
ua = vinterp(ncfile,ua,'p',levels,extrapolate=True)
va = vinterp(ncfile,va,'p',levels,extrapolate=True)
ua_backup = ua
va_backup =va

In [ ]:
data = xr.open_dataset(path)
#data.XTIME
data = data.sel(Time=timeidx)

In [ ]:
#Define constants
Rd = mpconst.dry_air_gas_constant.magnitude
kappa = mpconst.poisson_exponent.magnitude
cp = mpconst.Cp_d.magnitude
pref = mpconst.pot_temp_ref_press.magnitude * 100
g = mpconst.earth_gravity.magnitude
omega = 7.292E-5#mpconst.earth_avg_angular_vel.magnitude
r_earth = mpconst.earth_avg_radius.magnitude

In [ ]:
#Get lats and lons from wrf file
lats = getvar(ncfile, "lat").values[:,1]
lons = getvar(ncfile, "lon").values[1,:]

lons = np.deg2rad(lons)
lats = np.deg2rad(lats)

# Compute lat/lon deltas
dlon = np.diff(lons)
dlon2d = np.ones((lats.size, dlon.size)) * dlon

dlat = np.diff(lats)
dlat2d = np.ones((dlat.size, lons.size)) * np.expand_dims(dlat, axis=1)

# Convert deltas to physical distances
#earth_radius = data.LatLon_Projection.earth_radius * units.m

dx = r_earth * dlon2d * np.expand_dims(np.cos(lats), axis=1)
dy = r_earth * dlat2d
dx = dx[:, 1]
dy = dy[1, :]

In [ ]:
lat = np.rad2deg(lats)
lon = np.rad2deg(lons)
lat_rad = lats
lon_rad = lons
dlat = np.diff(lat)[0]
dlon = np.diff(lon)[0]
sigma2 = (dlon/dlat) ** 2
dellambda2 = (r_earth * dlat * (np.pi/180.))**2 

nz, ny, nx = hght.shape

In [ ]:
#Get sigma values from file and calculate pressure from sigma
sigma = ncfile.variables['ZNU'][:]
sigma = sigma[0] #clip to just one time step
modtop = 50. #hPa
surfp = 1000. #hPa
p = modtop + (sigma * (surfp - modtop))
p = p * 100
Z_sa, phi_sa, T_sa, S_half_sa = sa.atmos_structure(np.array(levels))

In [ ]:
f = 2 * omega * np.sin(lat_rad)
f[0] = f[1]
beta = 2 * omega / r_earth * np.cos(lat_rad)
#fo = 0.
#for j in range(lenlat):
#    fo = fo + (cor[j] / (dlat+1))
f0 = omega * (np.sin(lat_rad[-1]) + np.sin(lat_rad[0]))

phi_prime = g * hght - phi_sa[:, None, None]
tmpk_prime = tmpk - T_sa[:,None,None]

In [ ]:
def relative_vorticity(phi, lon, lat):
    
    
    # Define some constants
    #r_earth = 6371e3
    #omega = 7.292E-5
    lat_rad = np.deg2rad(lat)
    
    # Assume evenly grids are spaced
    dlat = lat[1] - lat[0]
    dlon = lon[1] - lon[0]   
    sigma2 = (dlon/dlat) ** 2
    dellambda2 = (r_earth * dlat * (np.pi/180.))**2 

    ######################## Calculate coefficients ########################
    # Note in this method, we didn't define the coefficient at the north and 
    # the south boundary, so Ai looks like [0, ..., 0]
    a1 = np.zeros(len(lat))
    a2 = np.zeros(len(lat))
    a3 = np.zeros(len(lat))
    a4 = np.zeros(len(lat))
    a5 = np.zeros(len(lat))

    for j in range(1, len(lat)-1):

        coslat_moh =  (np.cos(lat_rad[j]) + np.cos(lat_rad[j-1]))/2
        coslat_poh =  (np.cos(lat_rad[j]) + np.cos(lat_rad[j+1]))/2
        coslat = np.cos(lat_rad[j])

        
        a1[j] =  coslat_moh / coslat * sigma2
        a2[j] = 1 / (coslat ** 2)
        a3[j] = - ((2/(coslat ** 2)) + sigma2 / coslat * (coslat_moh + coslat_poh))
        a4[j] = a2[j]
        a5[j] = sigma2 * coslat_poh / coslat
        
    ######################## Calculate laplacien of geopotential ########################
    aphi = []
    for i in range(5):
        aphi.append(np.zeros(phi.shape))
        
    aphi[0][:, 1:, :]      = a1[None, 1:, None] * phi[:, :-1, :]    # i, j-1
    aphi[1][:, :, 1:]      = (a2[None, :, None] * phi)[:, :, :-1]         # i-1, j
    aphi[2][:, 1:-1, 1:-1] = (a3[None, :, None] * phi)[:, 1:-1, 1:-1] # i,j
    aphi[3][:, :, :-1]     = (a4[None, :, None] * phi)[:, :, 1:]         # i+1, j
    aphi[4][:, :-1, :]     = a5[None, :-1, None] * phi[:, 1:, :]   # i, j+1
    
    ######################## Convert geopotential to streamfunction ########################

    del2phi = sum(aphi) / dellambda2
    #del2psi = del2phi / f0
    
    return del2phi

In [ ]:
def stretching_term(phi, isob, S_half, f):
    #
    dp = isob[1:]- isob[:-1]
    dp = -dp
    dp = np.append(dp, dp[-1])
    
    # Use reciprocal of S
    S_half_r= 1/S_half
    # For purpose of calculate, append the last element of s_r to s+1/2, and the first ahead of s-1/2
    Sph_r = np.append(S_half_r, S_half_r[-1])
    Smh_r = np.append(S_half_r[0], S_half_r)

    ######################## Calculate Stretching term ########################
    # Again in this method, we didn't consider the top and the bottom 
    # boundary, so Ai looks like [0, ..., 0]
    a6phi = np.zeros(phi.shape)
    a3phi = np.zeros(phi.shape)
    a7phi = np.zeros(phi.shape)

    a3phi  = - (Smh_r + Sph_r)[:, None, None] * phi
    a6phi[1:, :, :]  =  Smh_r[1:, None, None] * phi[:-1, : ,:]
    a7phi[:-1, :, :] = Sph_r[:-1, None, None] * phi[1:, :, :]
    
    strat = (a3phi + a6phi + a7phi) * f[None, :, None] / (dp[:, None, None]**2)
    
    return strat

In [ ]:
dx = r_earth * dlon2d * np.expand_dims(np.cos(lats), axis=1)
dy = r_earth * dlat2d
print(ua[3,:,:].shape)
print(dx.shape)

ua = ua_backup.values
va = va_backup.values
ua = ua * units['m/s']
va = va * units['m/s']
dx = dx * units.meter
dy = dy * units.meter
rel_vor = np.zeros(ua.shape)
#https://gist.github.com/sgdecker/496f7ea7edd98b428ac3adab0b5e0842
for i in range(nz):
    rel_vor[i, :, :] = mpcalc.vorticity(ua[i,:,:], va[i,:,:], dx, dy, dim_order='yx')
#rel_vor = relative_vorticity(phi_prime, lon, lat)
stch = stretching_term(phi_prime, levels*100, S_half_sa, f)

In [ ]:
qgpv = stch + rel_vor + f[None, :, None]
plt.pcolormesh(lon, lat, qgpv[-5, :,:]*1e4,cmap="bwr",vmax=10,vmin=-10) #
#plt.pcolormesh(stch[10, :,:]) 
#plt.pcolormesh(lon[1:-1], Z_sa[7:-1], qgpv[7:-1,90,1:-1]) 
plt.title("cross section of q'")
plt.xlabel("longitude")
plt.ylabel("Altitude (m)")
plt.colorbar()

## Now we do some 3d inversion

In [ ]:
#nz, ny, nx = ua.shape
#phi_prime[18, 25:126,:101]
phi_prime_box = phi_prime#[:22, 25:127,:101]
tmpk_prime_box = tmpk_prime#[:22,25:127,:101]
#nx = 101
#ny = 102
#nz = nz
'''
set the lateral boundary conditions
'''
psi_ig = 10
psi = np.ones((nz,ny,nx)) * psi_ig
psi[:,:,0] = phi_prime_box[:,:,0] /f0
psi[:,:,-1] = phi_prime_box[:,:,-1] /f0
psi[:,0,:] = phi_prime_box[:,0,:] /f0
psi[:,-1,:] = phi_prime_box[:,-1,:] /f0

'''
consider temperature perturbations at the top and the surface
'''

tmpk_btm = (tmpk_prime_box[0, :, :] + tmpk_prime_box[1, :, :])/2
tmpk_top = (tmpk_prime_box[-1, :, :] + tmpk_prime_box[-1, :, :])/2

In [ ]:
'''
Calculate coefficients for inversion
coords: p:sfc->top, lon:W->E, lat: S->N
!!!! Change the order of index to z,lat, lon
'''
A1 = np.zeros(ny)
A2 = np.zeros(ny)
A3 = np.zeros(ny)
A4 = np.zeros(ny)
A5 = np.zeros(ny)
lat_rad_box = lat_rad#[25:127]
for j in range(1, ny-1):

    coslat_moh =  (np.cos(lat_rad_box[j]) + np.cos(lat_rad_box[j-1]))/2
    coslat_poh =  (np.cos(lat_rad_box[j]) + np.cos(lat_rad_box[j+1]))/2
    coslat = np.cos(lat_rad_box[j])


    A1[j] =  coslat_moh / coslat * sigma2
    A2[j] = 1 / (coslat ** 2)
    A3[j] = - ((2/(coslat ** 2)) + sigma2 / coslat * (coslat_moh + coslat_poh))
    A4[j] = A2[j]
    A5[j] = sigma2 * coslat_poh / coslat


pres = levels *100
dp = pres[1:] - pres[:-1]
dp = -dp
#dp = np.append(dp[0], dp)
dpph = np.append(dp, dp[-1])
dpmh = np.append(dp[0], dp)
dppmh = np.append(dp[0], dpph)
dppmh = (dppmh[1:] + dppmh[:-1])/2

#_, _, _, S_half_sa = sa.atmos_structure(pres)
S_half_r= 1/S_half_sa
# For purpose of calculate, append the last element of s_r to s+1/2, and the first ahead of s-1/2
Sph_r = np.append(S_half_r, S_half_r[-1])
Smh_r = np.append(S_half_r[0], S_half_r)
A3_s  = - (Smh_r + Sph_r) * f0**2 /dppmh**2 * (dellambda2)
A6 = Smh_r * f0**2 /dpmh**2 * (dellambda2)
A7 = Sph_r * f0**2 /dpph**2 * (dellambda2)


A3i_qgpv = np.zeros(phi_prime_box[:,:,0].shape)
A3i_qgpv = A3_s[:, None] + A3[None,:]

R = 287.05

In [ ]:
%%time
# SOR

threshold= 1.e-2        # value of error threshold
error_list = []
q_prime = qgpv.values
for iteration in range(500):
    if iteration%100 == 0:
        print(iteration)
    psi[0, :, :] = psi[1, :, :] - R / pres[0] * tmpk_btm * (-dp[0]) / f0
    psi[-1, :, :] = psi[-2, :, :] - R / pres[-1] * tmpk_top * dp[-1] / f0

    error = 0
    for k in range(1, nz-1):
        # check temperature bc later
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                # interior points
                lap = A1[j]*psi[k,j-1,i] + A2[j]*psi[k,j,i-1] + A3[j]*psi[k,j,i] + A4[j]*psi[k,j,i+1] + A5[j]*psi[k,j+1,i]      # calculation of residual
                stch_i= A6[k]*psi[k-1,j,i] + A3_s[k]*psi[k,j,i] + A7[k]*psi[k+1,j,i]
                res = lap + stch_i- q_prime[k,j,i]*dellambda2  # Why is the dellambda2 not appear in front of the stretching term
            
                #try:
                psi[k,j,i] = psi[k,j,i] - 1.9*res/A3i_qgpv[k,j]    # calculation of stream function
                #except IndexError:
                #    print(i, j ,k)
                error = error + abs(res)


        # if error threshold is met, break out of loop        
        unit_error = error/((nx-2)*(ny-2)*(nz-2))
        error_list.append(unit_error)
    if (unit_error < threshold):                
        print('amount of error:', iteration)
        print('got here: ', unit_error , iteration)
        break

        # number of iterations until convergence
print('number of iterations = ', iteration, unit_error)
#np.save('3d_ideal_psi',psi)

In [ ]:
lon_box = lon#[:101]
lat_box = lat#[25:127]
Z_sa_box = Z_sa#[:22]
# Plot figures
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,8), sharex=True)
cs = axes[0,1].contourf(lon_box, lat_box ,qgpv[10, :,:],cmap='PiYG') 
divider = make_axes_locatable(axes[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(cs, cax=cax, orientation='vertical')
#cs = axes[0].contour(xs[:,:,10],ys[:,:,10],q_prime[:,:,10],colors='black')
axes[0,1].set_title(r'$\frac{\partial \hat v}{\partial x} - \frac{\partial \hat v}{\partial x} + \frac{\partial}{\partial p} (R f_0 \hat \Theta)$')
axes[0,1].set_ylabel('latitude',fontsize=14)
    

psi_mean = np.mean(sens_array[:,:,1].flatten())
cs = axes[1,0].pcolormesh(lon_box, Z_sa, psi[:,110,:],cmap='bwr') 
axes[1,0].set_ylabel('Altitude',fontsize=14)
axes[1,0].set_xlabel('longitude',fontsize=14)
divider = make_axes_locatable(axes[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(cs, cax=cax, orientation='vertical')
#cs = axes[1,0].contour(xs[:,25,:],zs[:,25,:],psi[:,25,:],colors='k') 
axes[1,0].set_title('Sensitivity to PV vertical 43N crosssection')

cs = axes[0,0].pcolormesh(lon_box, Z_sa, qgpv[:,110,:],cmap='PiYG') 
divider = make_axes_locatable(axes[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(cs, cax=cax, orientation='vertical')
#cs = axes[0].contour(xs[:,:,10],ys[:,:,10],q_prime[:,:,10],colors='black')
axes[0,0].set_title(r'$\frac{\partial \hat v}{\partial x} - \frac{\partial \hat v}{\partial x} + \frac{\partial}{\partial p} (R f_0 \hat \Theta)$')
axes[0,0].set_ylabel('Altitude',fontsize=14)
     

cs = axes[1,1].pcolormesh(lon_box, lat_box ,psi[10,:,:],cmap='bwr') 
axes[1,1].set_ylabel('Latitude',fontsize=14)
axes[1,1].set_xlabel('Longitude',fontsize=14)
axes[1,1].axhline(y=lat[110])
divider = make_axes_locatable(axes[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(cs, cax=cax, orientation='vertical')
#cs = axes[1,1].contour(xs[:,:,10],ys[:,:,10],psi[:,:,10],colors='k') 
axes[1,1].set_title('Sensitivity to PV horizontal at 500hPa')

## And some 2d inversion

In [ ]:
#nz, ny, nx = ua.shape
#phi_prime[18, 25:126,:101]
phi_prime_box = phi_prime[10,:,:]#[:22, 25:127,:101]
tmpk_prime_box = tmpk_prime[10,:,:]#[:22,25:127,:101]
#nx = 101
#ny = 102
#nz = nz
'''
set the lateral boundary conditions
'''
psi_ig = 10
psi = np.ones((ny,nx)) * psi_ig
psi[:,0] = phi_prime_box[:,0] /f0
psi[:,-1] = phi_prime_box[:,-1] /f0
psi[0,:] = phi_prime_box[0,:] /f0
psi[-1,:] = phi_prime_box[-1,:] /f0

In [ ]:
%%time
# SOR

threshold= 1.e-2        # value of error threshold
error_list = []
q_prime = qgpv[10,:,:].values
for iteration in range(1000):
    if iteration%100 == 0:
        print(iteration)

    error = 0
    for j in range(1, ny-1):
        for i in range(1, nx-1):
            # interior points
            lap = A1[j]*psi[j-1,i] + A2[j]*psi[j,i-1] + A3[j]*psi[j,i] + A4[j]*psi[j,i+1] + A5[j]*psi[j+1,i]      # calculation of residual
            res = lap - q_prime[j,i]*dellambda2  # Why is the dellambda2 not appear in front of the stretching term

            #try:
            psi[j,i] = psi[j,i] - 1.9*res/A3[j]    # calculation of stream function
            #except IndexError:
            #    print(i, j ,k)
            error = error + abs(res)


    # if error threshold is met, break out of loop        
    unit_error = error/((nx-2)*(ny-2))
    error_list.append(unit_error)
    if (unit_error < threshold):                
        print('amount of error:', iteration)
        print('got here: ', unit_error , iteration)
        break

        # number of iterations until convergence
print('number of iterations = ', iteration, unit_error)